# Análise de segmentação de clientes usando RFM

## O que é?

A análise de RFM para segmentação de clientes é uma estratégia de marketing usada para mensurar o valor de um cliente para continuidade de um negócio, de acordo com três informações:

- Recência: o quão recentemente um cliente fez uma compra;
- Frequência: com que frequência o cliente faz uma compra; e
- Monetaridade: quanto dinheiro um cliente gasta em compras.

Essas três informações sobre um cliente 

In [100]:
# Importando as bibliotecas
%pip install --upgrade xlrd

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Note: you may need to restart the kernel to use updated packages.


In [101]:
# Verificando as versões das bibliotecas utilizadas
print('Versão do Pandas -> %s' % pd.__version__)
print('Versão do Numpy -> %s' % np.__version__)
print('Versão do Seaborn -> %s' % sns.__version__)

Versão do Pandas -> 1.3.4
Versão do Numpy -> 1.20.3
Versão do Seaborn -> 0.11.2


Importando os dados já previamente tratados no Notebook 01 - Tratamento e análise exploratória dos dados

In [102]:
df = pd.read_csv("./dados/dados-tratados.csv", sep=';')
df.head()

,N°_da_fatura,Data_da_fatura,ID_Cliente,País,Quantidade,Valor,Ano_mes,Pertence_UK
0,536365,2020-12-01 08:26:00,17850,United Kingdom,40,139.12,2020-12,SIM
1,536366,2020-12-01 08:28:00,17850,United Kingdom,12,22.20,2020-12,SIM
2,536368,2020-12-01 08:34:00,13047,United Kingdom,15,70.05,2020-12,SIM
3,536367,2020-12-01 08:34:00,13047,United Kingdom,83,278.73,2020-12,SIM
4,536369,2020-12-01 08:35:00,13047,United Kingdom,3,17.85,2020-12,SIM


In [103]:
# convertendo para o tipo data a coluna 'Data_da_fatura'
df['Data_da_fatura'] = pd.to_datetime(df['Data_da_fatura']).dt.date

# 1. Calculando a Recência, Frequência e Valor Monetário

## 1.1 Calculando a **Recência** dos clientes

In [104]:
df_recencia = df.groupby(by='ID_Cliente', as_index=False)['Data_da_fatura'].max()
df_recencia.columns = ['Cliente', 'Data_ultima_compra']

data_recencia = df_recencia['Data_ultima_compra'].max()
df_recencia['Recencia'] = df_recencia['Data_ultima_compra'].apply(lambda x: (data_recencia - x).days)
df_recencia.head()


,Cliente,Data_ultima_compra,Recencia
0,12346,2021-01-18,325
1,12347,2021-12-07,2
2,12348,2021-09-25,75
3,12349,2021-11-21,18
4,12350,2021-02-02,310


## 1.2 Calculando a **Frequência** dos clientes

In [105]:
df_frequencia = df.drop_duplicates().groupby(
    by=['ID_Cliente'], as_index=False)['Data_da_fatura'].count()
df_frequencia.columns = ['Cliente', 'Frequencia']
df_frequencia.head()


,Cliente,Frequencia
0,12346,1
1,12347,7
2,12348,4
3,12349,1
4,12350,1


## 1.3 Calculando o **Monetário** dos clientes

In [106]:
df_monetario = df.groupby(by='ID_Cliente', as_index=False)['Valor'].sum()
df_monetario.columns = ['Cliente', 'Monetario']
df_monetario.head()

,Cliente,Monetario
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40


In [107]:
df_rmf = df_recencia.merge(df_frequencia, on='Cliente')
df_rmf = df_rmf.merge(df_monetario, on='Cliente')

In [108]:
df.drop(columns=['N°_da_fatura', 'Data_da_fatura', 'Quantidade', 'Valor', 'Ano_mes', 'Pertence_UK'], inplace=True)
df.drop_duplicates(inplace=True)
df.rename(columns={'ID_Cliente':'Cliente'}, inplace=True)

In [109]:
df_rmf = df.merge(df_rmf, on='Cliente')
df_rmf

,Cliente,País,Data_ultima_compra,Recencia,Frequencia,Monetario
0,17850,United Kingdom,2020-12-02,372,34,5391.21
1,13047,United Kingdom,2021-11-08,31,10,3237.54
2,12583,France,2021-12-07,2,15,7281.38
3,13748,United Kingdom,2021-09-05,95,5,948.25
4,15100,United Kingdom,2021-01-10,333,3,876.00
...,...,...,...,...,...,...
4342,13436,United Kingdom,2021-12-08,1,1,196.89
4343,15520,United Kingdom,2021-12-08,1,1,343.50
4344,13298,United Kingdom,2021-12-08,1,1,360.00
4345,14569,United Kingdom,2021-12-08,1,1,227.39
